# Lecture 06

## Integration I:

### Introduction and Integration by Substitution

In [ ]:
import numpy as np
import sympy as sp
sp.init_printing()

##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from ipywidgets import IntSlider
from matplotlib import pyplot as plt

%matplotlib widget

global_fig_width = 9
global_fig_height = global_fig_width / 1.61803399
font_size = 10

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

## Warmup Exercise

$\displaystyle \frac{{\rm d}}{{\rm d}x} 3x^3 =$

$\displaystyle \frac{{\rm d}}{{\rm d}x} {4\over x^4} =$

$\displaystyle \frac{{\rm d}}{{\rm d}x} x^2\sin x =$

$\displaystyle \frac{{\rm d}}{{\rm d}t} \ln t =$

$\displaystyle \frac{{\rm d}}{{\rm d}t} e^{4t} =$

# Integration

**Calculating the area under a curve**

If we want to find the area under a curve, we can divide the area into strips, calculate the area of each strip, and sum these areas together.

e.g. find the area of

$$y = f(x) = x^2\qquad\mbox{between}\qquad x=1\text{ and }x=2$$


In [ ]:
import matplotlib.patches as patches

class InteractiveIntegrationSingle:
    def __init__(self):
        # Create figure and axes
        self.fig, self.ax = plt.subplots()
        # Hide the figure header and footer
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False

        # Plot settings
        self.ax.set_title('Integration Approximation with Single Rectangle')
        self.ax.set_xlabel('$x$')
        self.ax.set_ylabel('$f(x)$')
        self.ax.set_xlim(0.8, 2.2)
        self.ax.set_ylim(0, 5)

        # Plot the function f(x) = x^2
        x = np.linspace(0.8, 2.2, 1000)
        y = x**2
        self.line_f, = self.ax.plot(x, y, label='$f(x) = x^2$')

        # Plot x-axis
        self.ax.axhline(0, color='black', linewidth=1)

        # Plot vertical dashed lines at x=1 and x=2
        self.ax.plot([1, 1], [0, 1**2], ls='--', lw=2, color='gray')
        self.ax.plot([2, 2], [0, 2**2], ls='--', lw=2, color='gray')

        # Initialize the rectangle (patch)
        self.rect = patches.Rectangle((0, 0), 0, 0, facecolor='k', alpha=0.5)
        self.ax.add_patch(self.rect)

    def update(self, f_value):
        # Calculate rectangle parameters
        box = f_value
        box_width = 0.1  # Fixed width as in original code
        box_height = (box + box_width)**2

        # Update the rectangle
        self.rect.set_xy((box, 0))
        self.rect.set_width(box_width)
        self.rect.set_height(box_height)

        # Redraw the figure
        self.fig.canvas.draw_idle()

f_widget = FloatSlider(value=1.0, min=1.0, max=1.9, step=0.1)

In [ ]:
interactive_integration_single = InteractiveIntegrationSingle()
interact(interactive_integration_single.update, f_value=f_widget);

The area of the strip is $\delta A_i \approx y(x_i)~\delta x$

We can sum up all the strips between $x=1$ and $x=2$ to give an estimate for the total area:

$$A \approx \sum \delta A_i = \sum y(x_i)~\delta x.$$

This becomes more accurate as $\delta x$ becomes smaller.
We can use the same trick as when differentiating: let $\delta x$ tend to $0$:

$$A = \lim_{\delta x \to 0}\sum y(x_i)~\delta x
    = \int\limits^{2}_{1} y~{\rm d}x$$

$\int$ is the old English `S` and stands for the phrase "Sum Over".

This process is called **integration**.

In [ ]:
import matplotlib.patches as patches

class InteractiveIntegrationMulti:
    def __init__(self):
        # Create figure and axes
        self.fig, self.ax = plt.subplots()
        # Hide the figure header and footer
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False

        # Plot settings
        self.ax.set_title('Integration Approximation with Multiple Rectangles')
        self.ax.set_xlabel('$x$')
        self.ax.set_ylabel('$f(x)$')
        self.ax.set_xlim(0.8, 2.2)
        self.ax.set_ylim(0, 5)

        # Plot the function f(x) = x^2
        x = np.linspace(0.8, 2.2, 1000)
        y = x**2
        self.line_f, = self.ax.plot(x, y, label='$f(x) = x^2$')

        # Plot x-axis
        self.ax.axhline(0, color='black', linewidth=1)

        # Plot vertical dashed lines at x=1 and x=2
        self.ax.plot([1, 1], [0, 1**2], ls='--', lw=2, color='gray')
        self.ax.plot([2, 2], [0, 2**2], ls='--', lw=2, color='gray')

        # Initialize list to hold rectangle patches
        self.rects = []

    def update(self, n_boxes):
        # Remove previous rectangles
        for rect in self.rects:
            rect.remove()
        self.rects.clear()

        # Ensure n_boxes is at least 1 to avoid division by zero
        n_boxes = max(1, n_boxes)

        # Calculate box width
        box_width = (2 - 1) / n_boxes  # Interval length divided by number of boxes

        # Generate left edges of the boxes
        boxes = np.linspace(1, 2 - box_width, n_boxes)

        # Create new rectangles
        for box in boxes:
            # Calculate the height of the rectangle using right endpoints
            x_right = box + box_width
            box_height = x_right**2  # f(x_right)
            rect = patches.Rectangle((box, 0), box_width, box_height, facecolor='k', alpha=0.5)
            self.ax.add_patch(rect)
            self.rects.append(rect)

        # Redraw the figure
        self.fig.canvas.draw_idle()

n_widget = IntSlider(value=5, min=1, max=50, step=1, description='Num boxes')

In [ ]:
interactive_integration_multi = InteractiveIntegrationMulti()
interact(interactive_integration_multi.update, n_boxes=n_widget);

# Calculating the integral

Let us 'invent' a function $F(x)$ that gives the area from $0$ to $x$.
Then $A = F(2) - F(1)$.

With this (imagined) function, we can find the area of one of our tiny steps **exactly**, as

$$F(x + \delta x) - F(x)$$

Remember we approximated this as $\displaystyle y\,\delta x = f(x)\,\delta x$, so:

$$ F(x + \delta x) - F(x) \approx f(x)\delta x$$

$$ f(x) \approx {F(x+\delta x)-F(x)\over\delta x}$$

# Calculating the integral 
The error in this approximation tends to $0$ as $\delta x\to 0$, so

$$f(x) = \lim_{\delta x\to 0} {F(x+\delta x)-F(x)\over\delta x} = F'(x)$$

(recalling the definition of the derivative).

In other words, for our example,

$$A(x) = \int\limits^{x}_{1} y(t)~{\rm d}t\qquad\mbox{then}\qquad y(x) = A'(x)$$

**Integration reverses the process of differentiation**.

# Calculating integrals

We know that:
$$~~~~~~~~~~~~~~~~~~~~~~~{{\rm d} \left(x^2\right)\over {\rm d}x}=~2 x~~~~~\Rightarrow~~~~~
\biggr[x^2\biggl]_{x_1}^{x_2} = \int_{x_1}^{x_2} 2 x~{\rm d}x $$

Likewise:

$${{\rm d}\left(\frac{x^3}{3}\right)\over {\rm d}x}=~x^2~~~~~~~\Rightarrow~~~~~~
\int_{x=1}^{x=2} x^2~{\rm d}x = \biggr[{1\over
3}x^3\biggl]_{x=1}^{x=2}={8\over 3}-{1\over 3}= {7\over 3}$$

**In general:**

$${{\rm d}\left(a\thinspace x^{n+1}\right)\over {\rm d}x}=(n+1)a\thinspace~x^n~~~~~
\Rightarrow ~~~~\int_{x_1}^{x_2} a\thinspace x^n~{\rm d}x = \biggr[{a\over
(n+1)} \thinspace x^{(n+1)}\biggl]_{x_1}^{x_2}$$

# SymPy examples

$$\int_{x_1}^{x_2} x^6~{\rm d}x =$$

```python
x, x1, x2 = sp.symbols('x x_1 x_2')
sp.integrate(x**6,(x,x1,x2))
```

In [ ]:
x, x1, x2 = sp.symbols('x x_1 x_2')
sp.integrate(x**6,(x,x1,x2))

$$\int_{x_1}^{x_2} x^{3\over 2}~{\rm d}x =$$

```python
sp.integrate(x**(sp.sympify(3)/2),(x,x1,x2))
```

In [ ]:
sp.integrate(x**(sp.sympify(3)/2),(x,x1,x2))

$$\int_{x_1}^{x_2} x^{-{1\over 2}}~{\rm d}x = $$

```python
sp.integrate(x**(-sp.sympify(1)/2),(x,x1,x2))
```

In [ ]:
sp.integrate(x**(-sp.sympify(1)/2),(x,x1,x2))

# Indefinite integrals

Consider now the functions:

$$y=x^2+7 ~~~~~~~~~~{\rm (3)}$$

$$y=x^2-100 ~~~~~~~~~{\rm (4)}$$

Differentiating (3):
$${{\rm d}y\over {\rm d}x}=2x$$

Differentiating (4):
$${{\rm d}y\over {\rm d}x}=2x$$

This implies that the integral:

$$y=\int 2x~dx=x^2 + \kappa~~~~~{\rm where~~}\kappa~~{\rm is~an~unknown~constant}$$

An integral without limits is called an **indefinite integral**.

# Indefinite integrals in SymPy:

```python
x = sp.symbols('x')
sp.integrate(2*x,x)
```

In [ ]:
x = sp.symbols('x')
sp.integrate(2*x,x)

# Other integrals

recall that:

$${{\rm d}\over {\rm d}x}\ln x={1\over x}~~~~~~~~\Rightarrow~~\int{1\over x}~{\rm d}x=\ln x + 
\kappa$$

This constant $\kappa$ can be written as:

$$\kappa=\ln B$$
where $B$ is another constant. Thus:

$$\int {1\over x}~dx = \ln x + \kappa=\ln x +\ln B =\ln (B ~x)$$

# Integrating trigonometric functions

Recall that:

$${{\rm d}\over {\rm d}x}\biggr(\sin x\biggl)= \cos x~~~~~~~~~~~~~{\rm and}~~~~~~~~~~~
{{\rm d}\over {\rm d}x}\biggr(\cos x\biggl)= -\sin x$$ 

Example (i):
$$\int_{0}^{\pi/2} \cos x \thinspace dx =$$

```python
sp.integrate(sp.cos(x),(x,0,sp.pi/2))
```

In [ ]:
sp.integrate(sp.cos(x),(x,0,sp.pi/2))

  
Example (ii):
$$\int_{0}^{\pi/2} \sin x \thinspace {\rm d}x =$$
 

```python
sp.integrate(sp.sin(x),(x,0,sp.pi/2))
```

In [ ]:
sp.integrate(sp.sin(x),(x,0,sp.pi/2))

# Integrating trigonometric functions

Recall that:

$${{\rm d}\over {\rm d}x}\biggr(\sin x\biggl)= \cos x~~~~~~~~~~~~~{\rm and}~~~~~~~~~~~
{{\rm d}\over {\rm d}x}\biggr(\cos x\biggl)= -\sin x$$ 

Example (i):
$$\int_{0}^{\pi/2} \cos x \thinspace {\rm d}x =
  \biggr[\sin x \biggl]_0^{\pi/2}=1-0=1$$

Example (ii):
$$\int_{0}^{\pi/2} \sin x \thinspace {\rm d}x =
  \biggr[-\cos x \biggl]_0^{\pi/2}=0-(-1)=1$$

# Summary of integration formulae:

$\displaystyle \int a\,{\rm d}x=ax+C$

$\displaystyle \int ax\,{\rm d}x={1\over2}ax^2+C$

$\displaystyle \int ax^n\,{\rm d}x={1\over{n+1}}ax^{n+1}+C\qquad\mbox{for all }n\mbox{ except }n=-1$

$\displaystyle \int x^{-1}\,{\rm d}x=\int {1\over x}\,dx = \ln \vert x\vert +C$

# Application:

Recall that $f'(t)$ gives the rate at
which $f(t)$ changes at time $t$.  Integrating the derivative
$f'(t)$, we see:
$$\int_a^b f'(t) \thinspace {\rm d}t = \biggr[f(t)\biggl]_a^b = f(b)-f(a) $$

Therefore, the definite integral from $a$ to $b$ of $f'(t)$ with
respect to $t$ will always give the **net** change that $f(t)$ has
undergone as the parameter $t$ moves from $a$ to $b$.

## Example: 
A chemical process produces NaCl at the rate of $3\sqrt{t}$
grams per minute.  We ask three questions:

1. What is the rate of production one minute into the process?
2. What is the quantity of NaCl produced over the next three minutes?
3. What is the mean rate of NaCl production over this interval?

# Solution: 

Let $f(t)$ denote the grams of NaCl
produced after $t$ minutes.  Then $f'(t)= 3\sqrt{t}$.

1. The rate of production one minute into the process is $f'(1) = 3\sqrt{1} = 3$ grams per minute.

2. The quantity of NaCl produced over the next three minutes is
$$f(4)-f(1) = \int_1^4\,f'(t)\,{\rm d}t = \int_1^4\,3\sqrt{t}\,{\rm d}t
            = \phantom{\left[2t^{3/2}\right]_1^4 = 2\times(8-1) = 14}$$

```python
t = sp.Symbol('t')
sp.integrate(3*sp.sqrt(t),(t,1,4))
```

In [ ]:
t = sp.Symbol('t')
sp.integrate(3*sp.sqrt(t),(t,1,4))

(c) The mean rate is the constant rate which would give the same overall effect:
$${1\over b-a}\int_a^b\,f'(t)\,{\rm d}t = {1\over 4-1}\int_1^4\,3\sqrt{t}\,{\rm d}t
                                      = \phantom{{14\over 3}}$$

```python
sp.integrate(3*sp.sqrt(t),(t,1,4))/(4-1)
```

In [ ]:
sp.integrate(3*sp.sqrt(t),(t,1,4))/(4-1)

# Substitution Method

Suppose we wish to integrate: 

$$I=\int f(g(x))~{\rm d}x~~~~~~~~~~~{\rm~e.g.}~~\int_{a}^{b} (3-4x)^{-5}~{\rm d}x$$

In general, for $g(x)=u$,
we can write:

$$\int_{x_1}^{x_2}\thinspace f(g(x))~{\rm d}x =
\int_{u_1}^{u_2}\thinspace f(u) {{\rm d}x\over {\rm d}u} {\rm d}u $$

Integral version of the chain rule.

# Examples:

Consider:

$$\int_{a}^{b} (2x + 3)^4~{\rm d}x~~~~~~~~~~~~~{\rm Equation ~(7)}$$
Multiplying this out and then integrating it would be very tedious.
Try a substitution instead:

$$u=2x+3~~~\Longleftrightarrow x={1\over 2}(u-3)={u\over 2}-{3\over 2}~~~~
\Longleftrightarrow {{\rm d}x\over {\rm d}u}={1\over 2}$$ 

Substitute into Equation (7) above:

$$\int_{u(a)}^{u(b)} u^4 \times {1\over 2}~{\rm d}u ={1\over 2}\int_{u(a)}^{u(b)}
u^4~du = {1\over 2}\biggr[{1\over 5}u^5\biggl]_{u(a)}^{u(b)}=
{1\over 2}\biggr[{1\over 5}(2x+3)^5\biggl]_a^b$$ 

```python
a, b = sp.symbols('a b')
sp.integrate((2*x + 3)**4,(x,a,b))
```

In [ ]:
a, b = sp.symbols('a b')
sp.integrate((2*x + 3)**4,(x,a,b))

# Examples:

$$\int_{a}^{b} (3-4x)^{-5}~{\rm d}x~~~~~~~~~~~~~{\rm Equation~(8)}$$

Let:

$$u=3-4x~~~\Longleftrightarrow x={1\over 4}(3-u)={3\over 4}-{u\over 4}~~~~
\Longleftrightarrow {{\rm d}x\over {\rm d}u}=-{1\over 4}$$ 

Substitute into Equation (8) above:

\begin{align*}
\int_{u(a)}^{u(b)} u^{-5} \times {-1\over 4}~{\rm d}u =
-{1\over 4}\int_{u(a)}^{u(b)} u^{-5}~{\rm d}u &= -{1\over 4}\biggr[{1\over
-4}u^{-4}\biggl]_{u(a)}^{u(b)} \\
&= {1\over 16}\biggr[(3-4x)^{-4}\biggl]_a^b
\end{align*}

```python
sp.integrate((3 - 4*x)**(-5),(x,a,b))
```

In [ ]:
sp.integrate((3 - 4*x)**(-5),(x,a,b))